# Instalando Bibliotecas

In [1]:
from sqlalchemy import create_engine
from pandas import json_normalize
import pandas as pd
import json

## Conectando com o banco

In [2]:
oltp_engine = create_engine("sqlite:///OLTP.db")

## TABELAS DIMENSÃO

#### Resgatando query de marca e modelo

In [3]:
produto_query = '''
SELECT DISTINCT REPLACE(Mod.Descricao, ' ', '') || Mar.Descricao as ID_Produto
    , Mod.Descricao as Modelo
    , Mar.Descricao as Marca
FROM Modelo as Mod
JOIN Marca as Mar on Mod.ID_Marca = Mar.ID_Marca
'''

df_produto = pd.read_sql(produto_query, oltp_engine)
print(df_produto.head(5))

                     ID_Produto                    Modelo    Marca
0              iPhone13ProApple             iPhone 13 Pro    Apple
1  SamsungGalaxyS21UltraSamsung  Samsung Galaxy S21 Ultra  Samsung
2         GooglePixel6ProGoogle        Google Pixel 6 Pro   Google
3            OnePlus9ProOnePlus             OnePlus 9 Pro  OnePlus
4         XiaomiMi11UltraXiamoi        Xiaomi Mi 11 Ultra   Xiamoi


#### Resgatando query de cidade e estado

In [4]:
cep_query = '''
WITH CidadesCorrigidas AS (
    SELECT 
        CASE 
            WHEN Cid.Descricao = 'SP' THEN 'Sao Paulo'
            WHEN Cid.Descricao = 'SSA' THEN 'Salvador'
            WHEN Cid.Descricao = 'RJ' THEN 'Rio de Janeiro'
            ELSE Cid.Descricao
        END AS CidadeCorrigida
        , Est.Descricao AS Estado
    FROM Cidade AS Cid
    JOIN Estado AS Est ON Cid.ID_Estado = Est.ID_Estado
)
SELECT DISTINCT REPLACE(CidadeCorrigida, ' ', '') || REPLACE(Estado, ' ', '') AS ID_Cep
    , CidadeCorrigida AS Cidade
    , Estado
FROM CidadesCorrigidas;
'''

df_cep = pd.read_sql(cep_query, oltp_engine)
print(df_cep.head(5))

           ID_Cep      Cidade    Estado
0   RioBrancoAcre  Rio Branco      Acre
1   MaceioAlagoas      Maceio   Alagoas
2     MacapaAmapa      Macapa     Amapa
3  ManausAmazonas      Manaus  Amazonas
4   SalvadorBahia    Salvador     Bahia


#### Resgatando query de quadrimestre e ano

###### A função CAST((CAST... e strftime(... Pode ser suibstituida por EXTRACT ou DATEPART no SQL Server ou outras no Oracle

In [5]:
tempo_query = '''
WITH TempoCalculado AS (
    SELECT 
        CASE 
            WHEN strftime('%m', Data) BETWEEN '01' AND '04' THEN 1
            WHEN strftime('%m', Data) BETWEEN '05' AND '08' THEN 2
            WHEN strftime('%m', Data) BETWEEN '09' AND '12' THEN 3
        END AS Quadrimestre
        , strftime('%Y', Data) as Ano
    FROM Compra
)
SELECT DISTINCT Ano || Quadrimestre AS ID_Tempo
    , Quadrimestre
    , Ano
FROM TempoCalculado
'''

df_tempo = pd.read_sql(tempo_query, oltp_engine)
print(df_tempo.head(5))

  ID_Tempo  Quadrimestre   Ano
0    20243             3  2024
1    20231             1  2023
2    20242             2  2024
3    20221             1  2022
4    20232             2  2023


#### Resgatando cliente

In [6]:
cliente_query = '''
WITH CorrecaoCliente AS(
    SELECT ID_Cliente
        , CASE 
            WHEN Sexo = '2' THEN 'M'
            WHEN Sexo = '1' THEN 'F'
            ELSE Sexo
        END AS Sexo
        , Nome
    FROM Cliente
)
SELECT *
FROM CorrecaoCliente
'''

df_cliente_sql = pd.read_sql(cliente_query, oltp_engine)
print(df_cliente_sql)

    ID_Cliente Sexo             Nome
0            1    M    Joao da Silva
1            2    F   Maria Oliveira
2            3    M     Pedro Santos
3            4    F        Ana Costa
4            5    M  Lucas Rodrigues
..         ...  ...              ...
85          86    F  Patricia Santos
86          87    M   Lucas Oliveira
87          88    F     Juliana Lima
88          89    M     Pedro Mendes
89          90    F    Vanessa Costa

[90 rows x 3 columns]


In [7]:
with open(r'OLTP\Clientes_Dados_Complementar.json') as f:
    data = json.load(f)#### Resgatando query de cliente

df_cliente_json = json_normalize(data, 'Cliente', sep='_')
df_cliente_dimensao = df_cliente_json.drop(['Qtde_filhos','ID'],axis=1).drop_duplicates(subset=['Estado_civil'])
df_cliente_dimensao['Descricao'] = ['Solteiro', 'Casado'] 
print(df_cliente_dimensao.rename(columns={'Estado_civil': 'ID_Estado'}))

  ID_Estado Descricao
0         S  Solteiro
1         C    Casado


In [8]:
df_cliente_json = df_cliente_json.rename(columns={'ID': 'ID_Cliente'})

df_merged = pd.merge(df_cliente_sql, df_cliente_json, on="ID_Cliente", how="inner")
print(df_merged.head(5))

   ID_Cliente Sexo             Nome Estado_civil  Qtde_filhos
0           1    M    Joao da Silva            S            1
1           2    F   Maria Oliveira            C            2
2           3    M     Pedro Santos            C            4
3           4    F        Ana Costa            S            3
4           5    M  Lucas Rodrigues            C            1


## TABELAS FATO

#### Resgatando Concorrentes

In [9]:
df = pd.read_csv(r'.\OLTP\Concorrente.csv', encoding='latin-1', sep=';')
df["Quadrimestre"] = pd.cut(
    df["Mês"],
    bins=[0, 4, 8, 12],   # faixas: 1–4, 5–8, 9–12
    labels=[1, 2, 3]
).astype(int)
df["ID_Tempo"] = df["Ano"].astype(str) + df["Quadrimestre"].astype(str)

print(df.head(5).drop(columns=["Mês"]))

    Ano     Valor  Quantidade  Quadrimestre ID_Tempo
0  2021   3000,00           3             1    20211
1  2021   9700,00           8             1    20211
2  2021  13500,00          10             1    20211
3  2021  16700,00          14             1    20211
4  2021   2700,00           3             2    20212


#### Resgatando Histórico MimiPhones

In [10]:
mimi_query = '''
WITH CTE AS(
    SELECT REPLACE(Mod.Descricao, ' ', '') || Mar.Descricao as ID_Produto
        , Cli.ID_Cliente
        , Car.Valor
        , Car.Quantidade
        , CASE 
             WHEN Cid.Descricao = 'SP' THEN 'Sao Paulo'
             WHEN Cid.Descricao = 'SSA' THEN 'Salvador'
             WHEN Cid.Descricao = 'RJ' THEN 'Rio de Janeiro'
             ELSE Cid.Descricao
         END AS CidadeCorrigida
         , Est.Descricao AS Estado
         , CASE 
             WHEN strftime('%m', Data) BETWEEN '01' AND '04' THEN 1
             WHEN strftime('%m', Data) BETWEEN '05' AND '08' THEN 2
             WHEN strftime('%m', Data) BETWEEN '09' AND '12' THEN 3
         END AS Quadrimestre
         , strftime('%Y', Data) as Ano
    FROM Carrinho AS Car
    JOIN Aparelho AS Apa ON Apa.ID_Aparelho = Car.ID_Aparelho
    JOIN Modelo AS Mod ON Mod.ID_Modelo = Apa.ID_Modelo
    JOIN Marca AS Mar ON Mar.ID_Marca = Mod.ID_Marca
    JOIN Compra AS Com ON Com.ID_Compra = Car.ID_Compra
    JOIN Cliente as Cli ON Cli.ID_Cliente = Com.ID_Cliente
    JOIN Cidade as Cid ON Cid.ID_Cidade = Cli.ID_Cidade
    JOIN Estado as Est ON Est.ID_Estado = Cid.ID_Estado
)

SELECT DISTINCT REPLACE(CidadeCorrigida, ' ', '') || REPLACE(Estado, ' ', '') AS ID_Cep
    , Ano || Quadrimestre AS ID_Tempo
    , ID_Produto
    , ID_Cliente
    , Valor
    , Quantidade
FROM CTE 
'''

df_mimi = pd.read_sql(mimi_query, oltp_engine)
print(df_mimi.head(5))

                     ID_Cep ID_Tempo                       ID_Produto  \
0             SalvadorBahia    20243  SamsungGalaxyNote20UltraSamsung   
1             SalvadorBahia    20231               GooglePixel5Google   
2  RiodeJaneiroRiodeJaneiro    20242             MotorolaEdgeMotorola   
3        PortoVelhoRondonia    20221           MotorolaRazr5GMotorola   
4             SalvadorBahia    20232                       LGVelvetLG   

   ID_Cliente    Valor  Quantidade  
0          65  17583.3           3  
1          35  12381.6           1  
2          51   7334.7           1  
3          52   6714.0           1  
4          90  20823.6           1  


In [11]:
df_cliente = df_cliente_json.drop('Qtde_filhos', axis = 1)

df_mimiphone = pd.merge(df_mimi, df_cliente, on="ID_Cliente", how="inner")
print(df_mimiphone.drop('ID_Cliente', axis = 1))

                         ID_Cep ID_Tempo                       ID_Produto  \
0                 SalvadorBahia    20243  SamsungGalaxyNote20UltraSamsung   
1                 SalvadorBahia    20231               GooglePixel5Google   
2      RiodeJaneiroRiodeJaneiro    20242             MotorolaEdgeMotorola   
3            PortoVelhoRondonia    20221           MotorolaRazr5GMotorola   
4                 SalvadorBahia    20232                       LGVelvetLG   
...                         ...      ...                              ...   
10050             SalvadorBahia    20232            GooglePixel6ProGoogle   
10051        ParatyRiodeJaneiro    20221                  OnePlus9OnePlus   
10052               IlheusBahia    20232                    Nokia8.3Nokia   
10053           BoaVistaRoraima    20213                 XiaomiMi11Xiamoi   
10054              GoianiaGoias    20223           MotorolaRazr5GMotorola   

          Valor  Quantidade Estado_civil  
0      17583.30           3     

## PROVOANDO O BANCO OLAP

In [12]:
olap_engine = create_engine("sqlite:///OLAP.db")

In [13]:
df_produto.to_sql('Produto', olap_engine, if_exists='replace', index=False)

30

In [14]:
df_cep.to_sql('Lugar', olap_engine, if_exists='replace', index=False)

39

In [15]:
df_tempo.to_sql('Tempo', olap_engine, if_exists='replace', index=False)

15

In [16]:
df_cliente_dimensao.rename(columns={'Estado_civil': 'ID_Estado'}).to_sql('Estado Civil', olap_engine, if_exists='replace', index = False)

2

In [17]:
df.drop(columns=["Mês"]).to_sql('Concorrente', olap_engine, if_exists='replace', index = True, index_label ='ID_Concorrente')

48

In [18]:
df_mimiphone.rename(columns={'Estado_civil': 'ID_Estado'}).drop('ID_Cliente', axis = 1).to_sql('Venda', olap_engine, if_exists='replace', index = True, index_label='ID_Venda')

10055